In [5]:
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
import urllib.parse
import ssl

# SSL 에러 방지
context = ssl._create_unverified_context()

# 기존 데이터셋 로드
data = pd.read_csv('Go.csv', encoding='cp949')

# 맛집 정보를 담을 새로운 열 추가
data['맛집정보'] = ''

# 관광지명만 추출
tourist_spots = data['관광지명'].tolist()

# 관광지명을 URL에 적합하도록 인코딩
encoded_list = [urllib.parse.quote(s) for s in tourist_spots]

# 맛집 정보 가져오기
for spot, row_index in zip(encoded_list, range(len(encoded_list))):
    url = f'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={spot}+%EB%A7%9B%EC%A7%91'
    html = urllib.request.urlopen(url, context=context).read()
    soup = BeautifulSoup(html, 'html.parser')
    
    # 맛집명 + 카테고리 추출
    span_name = soup.find_all(class_='TYaxT')
    span_category = soup.find_all(class_='KCMnt')
    
    # 각 맛집명과 카테고리를 묶어서 맛집 정보 생성
    restaurant_info = []
    for name, category in zip(span_name, span_category):
        restaurant_info.append((name.text.strip(), category.text.strip()))
    
    # 데이터프레임에 맛집 정보 저장
    data.at[row_index, '맛집정보'] = restaurant_info

# 결과 출력
data.head()

HTTPError: HTTP Error 403: Forbidden

In [ ]:
num_rows = data.shape[0]
num_columns = data.shape[1]

print("행의 개수:", num_rows)
print("열의 개수:", num_columns)

In [ ]:
filtered_data = data[(data['KTO카테고리대분류명'] != '음식') & (data['KTO카테고리소분류명'] != '관광호텔') & (data['맛집정보'].apply(lambda x: len(x) > 0))]

In [ ]:
num_rows = filtered_data.shape[0]
num_columns = filtered_data.shape[1]

print("행의 개수:", num_rows)
print("열의 개수:", num_columns)

In [ ]:
# filtered_data.to_csv('Gangnam_new.csv', index=False, encoding='utf-8-sig')